In [1]:
import pandas as pd 
import numpy as np
import lime 
import lime.lime_tabular

from scipy import stats # библиотека статистических методов
from statsmodels.stats import diagnostic
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
#from sklearn.linear_model import LinearRegression
#from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_csv('cleared_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   4773 non-null   int64  
 1   program_city_group           3597 non-null   float64
 2   age_group                    4773 non-null   int64  
 3   service_type_id              4773 non-null   int64  
 4   distance_min                 2682 non-null   float64
 5   cpr_max                      3568 non-null   float64
 6   cpr_mean                     3568 non-null   float64
 7   number_of_clinics_in_progam  4773 non-null   int64  
 8   analysis_urin_avg_mean       2738 non-null   float64
 9   analysis_urin_avg_min        2738 non-null   float64
 10  analysis_copro_avg_min       2655 non-null   float64
 11  anti_hcv_avg_mean            2604 non-null   float64
 12  anti_hcv_avg_min             2604 non-null   float64
 13  doctor_visit_avg_m

Произведём разделение по возрастным группам

In [4]:
age_group1=df[df['age_group']==0]
age_group2=df[df['age_group']==1]
pd.concat([age_group1.target.describe().rename('Age 1'),
           age_group2.target.describe().rename('Age 2')],axis=1)

,Age 1,Age 2
count,3885.000000,888.000000
mean,20.233831,21.746013
std,20.862454,22.441655
min,0.113452,0.123416
25%,5.116960,5.473721
50%,13.519729,13.822131
75%,27.476425,29.241775
max,112.442635,111.517882


The second age group mean is slightly higher .

In [5]:
# Welch's test
stats.ttest_ind(age_group1.target, age_group2.target, equal_var=False)

Ttest_indResult(statistic=-1.834896688364854, pvalue=0.0667564825195631)

Достоверно нельзя сказать, что различие средних значений не случайно.

In [12]:
service_group1=df[df['service_type_id']==0]
service_group2=df[df['service_type_id']==1]
pd.concat([service_group1.target.describe().rename('Service type id 1'),
           service_group2.target.describe().rename('Service type id 2')],axis=1)

,Service type id 1,Service type id 2
count,2802.000000,1971.000000
mean,23.032718,16.936184
std,23.417821,16.856999
min,0.123416,0.113452
25%,5.259236,5.134283
50%,14.797439,12.351496
75%,33.265536,22.582651
max,112.442635,99.514563


In [7]:
# Welch's test
stats.ttest_ind(service_group1.target, service_group2.target, equal_var=False)

Ttest_indResult(statistic=10.457238637973528, pvalue=2.5458937763197536e-25)

Очевидно, что различие средних значений не случайно.

In [8]:
def selections(service_group):
    # Split (70% - training set, 30% - testing set)
    X, X_test, y, y_test = train_test_split(service_group.drop(columns='target'), service_group['target'], test_size = 0.3, random_state = 34)
    # Create an imputer object with a KNN strategy
    imputer = KNNImputer(n_neighbors=2, weights="uniform")
    # Train and transform the data
    imputer.fit(X)
    X = imputer.transform(X)
    X_test =imputer.transform(X_test)
    # Create the scaler object with a range of 0-1
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Train and transorm the data
    scaler.fit(X)
    X = scaler.transform(X)
    X_test = scaler.transform(X_test)
    
    # Mean Absolute Error
    def mae(y_true, y_pred):
        return np.mean(abs(y_true - y_pred))

    svm = SVR(C = 100, gamma = 0.1)
    # Train the model
    svm.fit(X, y)
    # Make predictions and predict
    svm_pred = svm.predict(X_test)
    svm_mae=mae(y_test, svm_pred)
    return svm_mae

In [9]:
mae=(selections(service_group1), selections(service_group2), selections(df))
mae

(15.374343327073694, 10.224875817315755, 13.71358849749915)

In [10]:
numshape=(service_group1.shape[0], service_group2.shape[0], df.shape[0])
numshape

(2802, 1971, 4773)

In [11]:
(numshape[2]*mae[2]-numshape[1]*mae[1]-numshape[0]*mae[0])/numshape[2]

0.46570661222996124

Разделение данных на выборки по service_group не привело к значительному изменению ошибки. Что немного парадоксально.